In [1]:
import os
import cv2
import albumentations as A
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split

## Transfer Learning to a YOLO (you only look once) model for Object Recognition

### Step 1: Resize all images to correct format

1. We must have all images in the correct format. Always this images have to be in a square format. In this case they going to ve 640x640 

In [2]:
FIRST_IMG_DIR = "YOLO"

SIZE = (640, 640)

os.makedirs(FIRST_IMG_DIR, exist_ok=True)

def resize_image(image_path: str, target_size: tuple[int,int]) -> np.ndarray:
    image = cv2.imread(image_path)
    if image is not None:
        resized_image = cv2.resize(image, target_size)
        return resized_image


for filename in os.listdir(FIRST_IMG_DIR):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(FIRST_IMG_DIR, filename)
        resized_image = resize_image(img_path, SIZE)

        if resized_image is not None:
            resized_image_path = os.path.join(FIRST_IMG_DIR, filename)
            cv2.imwrite(resized_image_path, resized_image)
            

### Step 2: Image Augmentation

The second step is to do data augmentation

In [ ]:
IMAGES_DIR = r'dataset\images'
LABELS_DIR = r'dataset\labels'

In [18]:

AUGMENTED_IMG_DIR = r'dataset/images_augmented'
AUGMENTED_LABELS_DIR = r'dataset/labels_augmented'

os.makedirs(AUGMENTED_IMG_DIR, exist_ok=True)
os.makedirs(AUGMENTED_LABELS_DIR, exist_ok=True)

def load_images_and_labels(images_dir: str, labels_dir: str) -> tuple[list, list]:
    images = []
    labels = []
    for filename in os.listdir(images_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(images_dir, filename)
            label_path = os.path.join(labels_dir, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
            image = cv2.imread(img_path)
            if image is not None:
                images.append(image)
                with open(label_path, 'r') as f:
                    labels.append(f.read())
    return images, labels


def save_augmented_data(image, label: str, image_name: str, label_name: str) -> None:
    cv2.imwrite(os.path.join(AUGMENTED_IMG_DIR, image_name), image)
    with open(os.path.join(AUGMENTED_LABELS_DIR, label_name), 'w') as f:
        f.write(label)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=10, p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


def augment_data(image: np.ndarray, label: str) -> tuple[np.ndarray, str]:
    boxes = []
    class_labels = []
    for line in label.split('\n'):
        if line.strip() == '':
            continue
        parts = line.strip().split()
        class_labels.append(int(parts[0]))
        boxes.append([float(x) for x in parts[1:]])

    bboxes = []
    for box in boxes:
        bboxes.append(box)

    augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    aug_image = augmented['image']
    aug_bboxes = augmented['bboxes']
    aug_class_labels = augmented['class_labels']

    aug_bboxes = [[max(0, min(1, coord)) for coord in bbox] for bbox in aug_bboxes]

    aug_label = ''
    for bbox, cls in zip(aug_bboxes, aug_class_labels):
        aug_label += f"{cls} " + " ".join(map(str, bbox)) + '\n'

    return aug_image, aug_label.strip()


images, labels = load_images_and_labels(IMAGES_DIR, LABELS_DIR)

for i, (image, label) in enumerate(zip(images, labels)):
    for j in range(5): 
        try:
            aug_image, aug_label = augment_data(image, label)
            image_name = f'aug_{i}_{j}.jpg'
            label_name = f'aug_{i}_{j}.txt'
            save_augmented_data(aug_image, aug_label, image_name, label_name, AUGMENTED_IMG_DIR, AUGMENTED_LABELS_DIR)
        except ValueError as e:
            print(f"Error: {e}")
            continue


### Step 3: Split in train and validation

In [19]:

TRAING_IMG_DIR = 'dataset/train/images'
TRAING_LABELS_DIR = 'dataset/train/labels'

VAL_IMG_DIR = 'dataset/val/images'
VAL_LABELS_DIR = 'dataset/val/labels'

os.makedirs(TRAING_IMG_DIR, exist_ok=True)
os.makedirs(TRAING_LABELS_DIR, exist_ok=True)
os.makedirs(VAL_IMG_DIR, exist_ok=True)
os.makedirs(VAL_LABELS_DIR, exist_ok=True)

images = [f for f in os.listdir(AUGMENTED_IMG_DIR) if f.endswith('.jpg') or f.endswith('.png')]
labels = [f.replace('.jpg', '.txt').replace('.png', '.txt') for f in images]

train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

def move_files(file_list: list[str], source_dir: str, dest_dir: str) -> None:
    for file_name in file_list:
        shutil.move(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

move_files(train_images, AUGMENTED_IMG_DIR, TRAING_IMG_DIR)
move_files(val_images, AUGMENTED_IMG_DIR, VAL_IMG_DIR)
move_files(train_labels, AUGMENTED_LABELS_DIR, TRAING_LABELS_DIR)
move_files(val_labels, AUGMENTED_LABELS_DIR, VAL_LABELS_DIR)
